In [1]:
# combine eia dfs into one
# reword eia df location lists
# scrape noted areas during outages + 1 month before & after
# nlp tweets, model outage probability
# plot outage-related tweets w/ probabilities

## Read in Data

In [2]:
import numpy as np
import pandas as pd

import time
import datetime

from twitterscraper import query_tweets

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13'}


In [3]:
# list of dfs of eia data
xlsx_list = [pd.read_excel(f'{year}.xlsx', header=1) 
              for year in range(2013,2020)]

In [4]:
# combine dfs of eia data
df = pd.concat(xlsx_list, sort=False)

### Reduce Data to just WA

In [5]:
# function to determine if event is in washington
def washington_check(loc):
    right_wa = 'Washington'
    wrong_wa = ('D.C', 'Columbia', 'DC', 
        'Washington County', 'Oregon Counties')
    if right_wa in loc and not any(wrong in loc for wrong in wrong_wa):
        return True
    return False

In [6]:
# washington df
wa = df[df['Area Affected'].fillna('?').map(washington_check)]

### Clean WA Data

In [7]:
# simplify/map area affected to simple wa counties separated by spaces
# WA for whole state and no more detail

wa_cty_dict = {
    'King, Pierce and Thurston Counties, Washington':\
        'King Pierce Thurston',
    'Alberta, Canada; Washington State':'WA',
    'King, Whatcom, and Skagit, Washington':'King Whatcom Skagit',
    'King County, Thurston County and Kitsap County,  Washington':\
        'King Thurston Kitsap',
    'Washington':'WA',
    'Kitsap, Thurston, Whatcom counties Washington':\
        'King Thurston Whatcom',
    'King County, Skagit County, Whatcom County, Kitsap County, '+\
        'Pierce County, Thurston County, Island County, Washington':\
        'King Skagit Whatcom Kitsap Pierce Thurston Island',
    'King County, Washington':'King',
    'Washington: Snohomish County;':'Snohomish',
    'Washington: Stevens County, Lincoln County, Adams County, Whitman '+\
        'County, Spokane County; Idaho: Bonner County, Kootenai County, '+\
        'Shoshone County, Benewah County, Idaho County;':\
        'Stevens Lincoln Adams Whitman Spokane',
    'Washington: King County, Whatcom County, Kitsap County, '+\
        'Skagit County;':\
        'King Whatcom Kitsap Skagit',
    'Washington: Clark County;':'Clark',
    'Washington: King County;':'King',
    'Washington:':'WA',
    'Washington: Montana:':'WA',
    'Washington: Whatcom County; Montana:':'Whatcom',
    'Washington: Island County, King County, Kitsap County, Thurston '+\
        'County, Skagit County, Whatcom County;':\
        'Island King Kitsap Thurston Skagit Whatcom',
    'Washington: King County, Kitsap County, Island County, Pierce '+\
        'County, Thurston County, Whatcom County, Skagit County, '+\
        'Kittitas County;':\
        'King Kitsap Island Pierce Thurston Whatcom Skagit Kittitas',
    'Washington: Skagit County, Snohomish County, King County, Kitsap '+\
        'County, Island County;':\
        'Skagit Snohomish King Kitsap Island'
}

In [8]:
# format Area Affected

wa.loc[:,'Area Affected'] = wa['Area Affected'].map(
    wa_cty_dict).apply(lambda s:s.split())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
wa = wa.reset_index()

# remove ongoings and . . s from restoration d&t
rem_ong = lambda t: np.nan if ('ongoing' in t) or \
    ('.        .' in t) else t
wa.loc[:,'Restoration Date and Time'] = \
    wa['Restoration Date and Time'].map(rem_ong)

In [10]:
# adapt for new nans in restoration d&t

for row in wa.index:
    if (wa.loc[row, 'Restoration Date and Time'] is np.nan):
        wa.loc[row, 'Restoration Date and Time'] = \
            wa.loc[row, 'Event Date and Time']

In [11]:
# to_datetime date and time values
wa.loc[:,'Event Date and Time'] = pd.to_datetime(wa['Event Date and Time'])
wa.loc[:,'Restoration Date and Time'] = \
    pd.to_datetime(wa['Restoration Date and Time'])

In [12]:
# reorder columns for convenience in next block 

cols = list(wa.columns)
cols.remove('Area Affected')
wa = wa[['Area Affected']+cols]

In [13]:
# multiply rows with multiple affected counties

# adapted from piRSquared at stackoverflow.com/questions/50217968
wa = pd.DataFrame([
    [aa] + _ for AA, *_ in wa.values
    for aa in AA
], columns=wa.columns)

In [14]:
# save wa

wa.to_csv('washington_outages.csv', index=False)

In [15]:
# format query

def custom_query(near, begin=None, end=None, terms = '', pools=10):
    
    time.sleep(1)
    
    if 'WA' not in near:
        near += ' County, WA' # format counties properly
    
    if begin != None and end != None:
        begin = begin.to_pydatetime().date()
        end = end.to_pydatetime().date() + datetime.timedelta(days=1)
    else:
        begin = datetime.date(2012,1,1)
        end = datetime.date(2019,12,31)
        
    # account for short outages, aftertalk
        
    tweets = query_tweets(f'{terms} near:"{near}"', pools,
                          begindate=begin, enddate=end)
    
    return tweets

In [16]:
# query for each out county for each outage time

tweets_dict = {} # key is index of row in wa df
for row in wa.index:
    
    near = wa.loc[row, 'Area Affected']
    begin = wa.loc[row, 'Event Date and Time']
    end = wa.loc[row, 'Restoration Date and Time']
    
    tweets = custom_query(near, begin, end, 
                          'blackout OR outage OR power', 3)
    tweets_dict[row] = tweets
    
    print(row)

INFO: queries: ['blackout OR outage OR power near:"King County, WA" since:2012-01-19 until:2012-01-20', 'blackout OR outage OR power near:"King County, WA" since:2012-01-20 until:2012-01-21']
INFO: Got 12 tweets (12 new).
INFO: Got 28 tweets (16 new).


0


INFO: queries: ['blackout OR outage OR power near:"Pierce County, WA" since:2012-01-19 until:2012-01-20', 'blackout OR outage OR power near:"Pierce County, WA" since:2012-01-20 until:2012-01-21']
INFO: Got 20 tweets (20 new).
INFO: Got 40 tweets (20 new).


1


INFO: queries: ['blackout OR outage OR power near:"Thurston County, WA" since:2012-01-19 until:2012-01-20', 'blackout OR outage OR power near:"Thurston County, WA" since:2012-01-20 until:2012-01-21']
INFO: Got 19 tweets (19 new).
INFO: Got 33 tweets (14 new).


...

70


INFO: queries: ['blackout OR outage OR power near:"Kitsap County, WA" since:2018-12-20 until:2018-12-21']
INFO: Got 6 tweets (6 new).


71


INFO: queries: ['blackout OR outage OR power near:"Island County, WA" since:2018-12-20 until:2018-12-21']
INFO: Got 0 tweets (0 new).


72


In [17]:
# assemble tweet info into dataframe

df_list = []

for k,v in tweets_dict.items():
    _df = pd.DataFrame({'event_row_or_county':k,
                        'timestamp':[t.timestamp for t in v],
                        'text':[t.text for t in v],
                        'outage':1})
    df_list.append(_df)
    
tweets_df_1 = pd.concat(df_list)

Also want a generic corpus to train on.

In [18]:
# query for each county just over time, not outages

tweets_dict = {} # key is index of row in wa df
for cty in wa['Area Affected'].unique(): # all counties in wa w/ outages
    
    near = cty
    
    tweets = custom_query(near, None, None, '', 3)
    tweets_dict[cty] = tweets
    
    print(cty)

INFO: queries: [' near:"King County, WA" since:2012-01-01 until:2012-05-26', ' near:"King County, WA" since:2012-05-26 until:2012-10-19', ' near:"King County, WA" since:2012-10-19 until:2013-03-14', ' near:"King County, WA" since:2013-03-14 until:2013-08-07', ' near:"King County, WA" since:2013-08-07 until:2013-12-31', ' near:"King County, WA" since:2013-12-31 until:2014-05-26', ' near:"King County, WA" since:2014-05-26 until:2014-10-19', ' near:"King County, WA" since:2014-10-19 until:2015-03-14', ' near:"King County, WA" since:2015-03-14 until:2015-08-07', ' near:"King County, WA" since:2015-08-07 until:2015-12-31', ' near:"King County, WA" since:2015-12-31 until:2016-05-25', ' near:"King County, WA" since:2016-05-25 until:2016-10-18', ' near:"King County, WA" since:2016-10-18 until:2017-03-13', ' near:"King County, WA" since:2017-03-13 until:2017-08-06', ' near:"King County, WA" since:2017-08-06 until:2017-12-30', ' near:"King County, WA" since:2017-12-30 until:2018-05-25', ' near:"

King


INFO: queries: [' near:"Pierce County, WA" since:2012-01-01 until:2012-05-26', ' near:"Pierce County, WA" since:2012-05-26 until:2012-10-19', ' near:"Pierce County, WA" since:2012-10-19 until:2013-03-14', ' near:"Pierce County, WA" since:2013-03-14 until:2013-08-07', ' near:"Pierce County, WA" since:2013-08-07 until:2013-12-31', ' near:"Pierce County, WA" since:2013-12-31 until:2014-05-26', ' near:"Pierce County, WA" since:2014-05-26 until:2014-10-19', ' near:"Pierce County, WA" since:2014-10-19 until:2015-03-14', ' near:"Pierce County, WA" since:2015-03-14 until:2015-08-07', ' near:"Pierce County, WA" since:2015-08-07 until:2015-12-31', ' near:"Pierce County, WA" since:2015-12-31 until:2016-05-25', ' near:"Pierce County, WA" since:2016-05-25 until:2016-10-18', ' near:"Pierce County, WA" since:2016-10-18 until:2017-03-13', ' near:"Pierce County, WA" since:2017-03-13 until:2017-08-06', ' near:"Pierce County, WA" since:2017-08-06 until:2017-12-30', ' near:"Pierce County, WA" since:2017-1

Pierce


INFO: queries: [' near:"Thurston County, WA" since:2012-01-01 until:2012-05-26', ' near:"Thurston County, WA" since:2012-05-26 until:2012-10-19', ' near:"Thurston County, WA" since:2012-10-19 until:2013-03-14', ' near:"Thurston County, WA" since:2013-03-14 until:2013-08-07', ' near:"Thurston County, WA" since:2013-08-07 until:2013-12-31', ' near:"Thurston County, WA" since:2013-12-31 until:2014-05-26', ' near:"Thurston County, WA" since:2014-05-26 until:2014-10-19', ' near:"Thurston County, WA" since:2014-10-19 until:2015-03-14', ' near:"Thurston County, WA" since:2015-03-14 until:2015-08-07', ' near:"Thurston County, WA" since:2015-08-07 until:2015-12-31', ' near:"Thurston County, WA" since:2015-12-31 until:2016-05-25', ' near:"Thurston County, WA" since:2016-05-25 until:2016-10-18', ' near:"Thurston County, WA" since:2016-10-18 until:2017-03-13', ' near:"Thurston County, WA" since:2017-03-13 until:2017-08-06', ' near:"Thurston County, WA" since:2017-08-06 until:2017-12-30', ' near:"T

...

Spokane


INFO: queries: [' near:"Clark County, WA" since:2012-01-01 until:2012-05-26', ' near:"Clark County, WA" since:2012-05-26 until:2012-10-19', ' near:"Clark County, WA" since:2012-10-19 until:2013-03-14', ' near:"Clark County, WA" since:2013-03-14 until:2013-08-07', ' near:"Clark County, WA" since:2013-08-07 until:2013-12-31', ' near:"Clark County, WA" since:2013-12-31 until:2014-05-26', ' near:"Clark County, WA" since:2014-05-26 until:2014-10-19', ' near:"Clark County, WA" since:2014-10-19 until:2015-03-14', ' near:"Clark County, WA" since:2015-03-14 until:2015-08-07', ' near:"Clark County, WA" since:2015-08-07 until:2015-12-31', ' near:"Clark County, WA" since:2015-12-31 until:2016-05-25', ' near:"Clark County, WA" since:2016-05-25 until:2016-10-18', ' near:"Clark County, WA" since:2016-10-18 until:2017-03-13', ' near:"Clark County, WA" since:2017-03-13 until:2017-08-06', ' near:"Clark County, WA" since:2017-08-06 until:2017-12-30', ' near:"Clark County, WA" since:2017-12-30 until:2018-

Clark


INFO: queries: [' near:"Kittitas County, WA" since:2012-01-01 until:2012-05-26', ' near:"Kittitas County, WA" since:2012-05-26 until:2012-10-19', ' near:"Kittitas County, WA" since:2012-10-19 until:2013-03-14', ' near:"Kittitas County, WA" since:2013-03-14 until:2013-08-07', ' near:"Kittitas County, WA" since:2013-08-07 until:2013-12-31', ' near:"Kittitas County, WA" since:2013-12-31 until:2014-05-26', ' near:"Kittitas County, WA" since:2014-05-26 until:2014-10-19', ' near:"Kittitas County, WA" since:2014-10-19 until:2015-03-14', ' near:"Kittitas County, WA" since:2015-03-14 until:2015-08-07', ' near:"Kittitas County, WA" since:2015-08-07 until:2015-12-31', ' near:"Kittitas County, WA" since:2015-12-31 until:2016-05-25', ' near:"Kittitas County, WA" since:2016-05-25 until:2016-10-18', ' near:"Kittitas County, WA" since:2016-10-18 until:2017-03-13', ' near:"Kittitas County, WA" since:2017-03-13 until:2017-08-06', ' near:"Kittitas County, WA" since:2017-08-06 until:2017-12-30', ' near:"K

Kittitas


In [19]:
# assemble tweet info into dataframe

df_list = []

for k,v in tweets_dict.items():
    _df = pd.DataFrame({'event_row_or_county':k,
                        'timestamp':[t.timestamp for t in v],
                        'text':[t.text for t in v],
                        'outage':0})
    df_list.append(_df)
    
tweets_df_0 = pd.concat(df_list)

In [63]:
# query for each county just over time, unlisted potential outages

tweets_dict = {} # key is index of row in wa df
for cty in wa['Area Affected'].unique(): # all counties in wa w/ outages
    
    near = cty
    
    tweets = custom_query(near, None, None, 
                          'outage OR blackout OR power', 3)
    tweets_dict[cty] = tweets
    
    print(cty)

INFO: queries: ['outage OR blackout OR power near:"King County, WA" since:2012-01-01 until:2012-05-26', 'outage OR blackout OR power near:"King County, WA" since:2012-05-26 until:2012-10-19', 'outage OR blackout OR power near:"King County, WA" since:2012-10-19 until:2013-03-14', 'outage OR blackout OR power near:"King County, WA" since:2013-03-14 until:2013-08-07', 'outage OR blackout OR power near:"King County, WA" since:2013-08-07 until:2013-12-31', 'outage OR blackout OR power near:"King County, WA" since:2013-12-31 until:2014-05-26', 'outage OR blackout OR power near:"King County, WA" since:2014-05-26 until:2014-10-19', 'outage OR blackout OR power near:"King County, WA" since:2014-10-19 until:2015-03-14', 'outage OR blackout OR power near:"King County, WA" since:2015-03-14 until:2015-08-07', 'outage OR blackout OR power near:"King County, WA" since:2015-08-07 until:2015-12-31', 'outage OR blackout OR power near:"King County, WA" since:2015-12-31 until:2016-05-25', 'outage OR black

King


INFO: queries: ['outage OR blackout OR power near:"Pierce County, WA" since:2012-01-01 until:2012-05-26', 'outage OR blackout OR power near:"Pierce County, WA" since:2012-05-26 until:2012-10-19', 'outage OR blackout OR power near:"Pierce County, WA" since:2012-10-19 until:2013-03-14', 'outage OR blackout OR power near:"Pierce County, WA" since:2013-03-14 until:2013-08-07', 'outage OR blackout OR power near:"Pierce County, WA" since:2013-08-07 until:2013-12-31', 'outage OR blackout OR power near:"Pierce County, WA" since:2013-12-31 until:2014-05-26', 'outage OR blackout OR power near:"Pierce County, WA" since:2014-05-26 until:2014-10-19', 'outage OR blackout OR power near:"Pierce County, WA" since:2014-10-19 until:2015-03-14', 'outage OR blackout OR power near:"Pierce County, WA" since:2015-03-14 until:2015-08-07', 'outage OR blackout OR power near:"Pierce County, WA" since:2015-08-07 until:2015-12-31', 'outage OR blackout OR power near:"Pierce County, WA" since:2015-12-31 until:2016-05

Pierce


INFO: queries: ['outage OR blackout OR power near:"Thurston County, WA" since:2012-01-01 until:2012-05-26', 'outage OR blackout OR power near:"Thurston County, WA" since:2012-05-26 until:2012-10-19', 'outage OR blackout OR power near:"Thurston County, WA" since:2012-10-19 until:2013-03-14', 'outage OR blackout OR power near:"Thurston County, WA" since:2013-03-14 until:2013-08-07', 'outage OR blackout OR power near:"Thurston County, WA" since:2013-08-07 until:2013-12-31', 'outage OR blackout OR power near:"Thurston County, WA" since:2013-12-31 until:2014-05-26', 'outage OR blackout OR power near:"Thurston County, WA" since:2014-05-26 until:2014-10-19', 'outage OR blackout OR power near:"Thurston County, WA" since:2014-10-19 until:2015-03-14', 'outage OR blackout OR power near:"Thurston County, WA" since:2015-03-14 until:2015-08-07', 'outage OR blackout OR power near:"Thurston County, WA" since:2015-08-07 until:2015-12-31', 'outage OR blackout OR power near:"Thurston County, WA" since:20

...

Spokane


INFO: queries: ['outage OR blackout OR power near:"Clark County, WA" since:2012-01-01 until:2012-05-26', 'outage OR blackout OR power near:"Clark County, WA" since:2012-05-26 until:2012-10-19', 'outage OR blackout OR power near:"Clark County, WA" since:2012-10-19 until:2013-03-14', 'outage OR blackout OR power near:"Clark County, WA" since:2013-03-14 until:2013-08-07', 'outage OR blackout OR power near:"Clark County, WA" since:2013-08-07 until:2013-12-31', 'outage OR blackout OR power near:"Clark County, WA" since:2013-12-31 until:2014-05-26', 'outage OR blackout OR power near:"Clark County, WA" since:2014-05-26 until:2014-10-19', 'outage OR blackout OR power near:"Clark County, WA" since:2014-10-19 until:2015-03-14', 'outage OR blackout OR power near:"Clark County, WA" since:2015-03-14 until:2015-08-07', 'outage OR blackout OR power near:"Clark County, WA" since:2015-08-07 until:2015-12-31', 'outage OR blackout OR power near:"Clark County, WA" since:2015-12-31 until:2016-05-25', 'outa

Clark


INFO: queries: ['outage OR blackout OR power near:"Kittitas County, WA" since:2012-01-01 until:2012-05-26', 'outage OR blackout OR power near:"Kittitas County, WA" since:2012-05-26 until:2012-10-19', 'outage OR blackout OR power near:"Kittitas County, WA" since:2012-10-19 until:2013-03-14', 'outage OR blackout OR power near:"Kittitas County, WA" since:2013-03-14 until:2013-08-07', 'outage OR blackout OR power near:"Kittitas County, WA" since:2013-08-07 until:2013-12-31', 'outage OR blackout OR power near:"Kittitas County, WA" since:2013-12-31 until:2014-05-26', 'outage OR blackout OR power near:"Kittitas County, WA" since:2014-05-26 until:2014-10-19', 'outage OR blackout OR power near:"Kittitas County, WA" since:2014-10-19 until:2015-03-14', 'outage OR blackout OR power near:"Kittitas County, WA" since:2015-03-14 until:2015-08-07', 'outage OR blackout OR power near:"Kittitas County, WA" since:2015-08-07 until:2015-12-31', 'outage OR blackout OR power near:"Kittitas County, WA" since:20

Kittitas


In [65]:
# assemble tweet info into dataframe

df_list = []

for k,v in tweets_dict.items():
    _df = pd.DataFrame({'event_row_or_county':k,
                        'timestamp':[t.timestamp for t in v],
                        'text':[t.text for t in v],
                        'outage':-1}) # don't know!
    df_list.append(_df)
    
tweets_df_n1 = pd.concat(df_list)

In [66]:
tweets_df = pd.concat([tweets_df_1, tweets_df_0, tweets_df_n1]
                     ).reset_index(drop=True)

In [67]:
# save tweets dataframe

tweets_df.to_csv('washington_outage_tweets.csv', index=False)